In [1]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import pandas as pd
import json
import numpy as np

try:
    HERE = Path(__file__).resolve()
except NameError:
    HERE = Path.cwd()

data = pd.read_csv("final.csv", dtype=str)
data.head(10)

,Unnamed: 0,ingredient,amount,ingr_annotation,amount_annotation,norm_value,norm_unit,nutrition,ingr_json,amount_json
0,15,bio hähnchenschlegel,4 stk.,"{""anzahl"": 4, ""einheit"": ""stk.""}","{""zutat"": ""H\u00e4hnchenschlegel"", ""eigenschaf...",4.0,stück,NaN,"[{'ingredient': 'Hähnchen', 'properties': ['Bi...","[{'amount': '4', 'unit': 'Stück'}]"
1,31,schokoriegel,1 stk.,"{""anzahl"": 1, ""einheit"": ""stk.""}","{""zutat"": ""Schokoriegel""}",1.0,stück,NaN,[{'ingredient': 'Schokoriegel'}],"[{'amount': 1, 'unit': 'Stück'}]"
2,32,amarettini mandelkeksezerstoßen,2 esslöffel,"{""anzahl"": 2, ""einheit"": ""essl\u00f6ffel""}","{""zutat"": ""Amarettini"", ""eigenschaft"": ""zersto...",0.03,liter,NaN,"[{'ingredient': 'Amarettini', 'properties': ['...","[{'amount': 2, 'unit': 'Esslöffel'}]"
3,33,deko schirmchen,2 stk.,"{""anzahl"": 2, ""einheit"": ""stk.""}","{""zutat"": ""Schirmchen"", ""eigenschaft"": ""Deko""}",2.0,stück,NaN,[],"[{'amount': '2', 'unit': 'Stück'}]"
4,40,chilisoße sambal oelek,0.5 tl,"{""volumen"": 0.5, ""einheit"": ""tl""}","{""zutat"": ""Chiliso\u00dfe"", ""eigenschaft"": ""Sa...",0.0025,liter,NaN,"[{'ingredient': 'Chili-Sauce', 'properties': [...","[{'amount': 0.5, 'unit': 'teelöffel'}]"
5,46,orangenabrieb,1 teelöffel,"{""volumen"": ""1"", ""einheit"": ""teel\u00f6ffel""}","{""zutat"": ""Teel\u00f6ffel"", ""eigenschaft"": ""or...",0.005,liter,NaN,"[{'ingredient': 'Orange', 'properties': ['Abri...","[{'amount': 1, 'unit': 'teelöffel'}]"
6,52,datteltomaten,4 stk.,"{""anzahl"": 4, ""einheit"": ""stk.""}","{""zutat"": ""Datteltomate"", ""eigenschaft"": ""ganz""}",4.0,stück,NaN,"[{'ingredient': 'Tomate', 'properties': ['Datt...","[{'amount': 4, 'unit': 'Stück'}]"
7,65,riesling trocken,100 ml,"{""volumen"": 100, ""einheit"": ""ml""}","{""zutat"": ""Riesling"", ""eigenschaft"": ""trocken""}",0.1,liter,NaN,"[{'ingredient': 'Wein', 'properties': ['trocke...","[{'volume': 100, 'unit': 'ml'}]"
8,103,schinkenwürfel,250 g,"{""gewicht"": 250, ""einheit"": ""g""}","{""zutat"": ""Schinkenw\u00fcrfel""}",250.0,g,NaN,"[{'ingredient': 'Schinken', 'properties': ['ge...","[{'weight': 250, 'unit': 'g'}]"
9,127,asiatische fischsoße,1 el,"{""volumen"": ""1"", ""einheit"": ""el""}","{""zutat"": ""Fischso\u00dfe"", ""eigenschaft"": ""as...",0.015,liter,NaN,[],"[{'amount': 1, 'unit': 'Esslöffel'}]"


In [ ]:
from loguru import logger

logger.add("API_Errors.log", level = "INFO", format = "{time}:{level}:{message}")

In [ ]:
import requests

url = "https://smarthome.uni-regensburg.de/naehrwertrechner/api/1.0/recipe_info_optifast"

def get_nut(prompt: str) -> dict | None:
    payload = { "recipe": prompt }
    headers = {"content-type": "application/json"}
    response = requests.post(url, json = payload, headers = headers, verify=False)

    if response.ok:
        return response.json()
    else:
        return None